# Demo of a simple SOSP model of classic agreement attraction

## Introduction
Starting with a featural lexical-dependency grammar, we design a Harmony Function (HF).  The function is defined on a space of link and treelet feature variables.  The gradient of HF gives the dynamics of processing, i.e., it specifies the way the link and feature values change from any initial state.  Sentence comprehension consists of (c1) starting with the system in an initial state in which all variables have the value 0, (c2) perceiving a word, which results in the setting of a subset of treelet features to non-zero values, (c3) gravitating under the dynamics + noise to a low velocity threshold (this generally results in nearly reaching a local harmony peak), and repeating from step (c2) until all the words in the sequence to be comprehended have been processed.  Random production (useful for probing the non-contextualized distribution over parse trajectories) is accomplished by (p1) starting in the zero state, (p2) using the prior of initial word likelihoods to select a word, (p3) activating the treelet features associated with that word, (p4) gravitating under the dynamics + noise to the low velocity threshold, (p5) generating a distribution over next words from the ensemble of open lexical attachment site activations, (p6) generating a new word, and repeating from step (p3) until (p4) has produced maximal harmony or a practical limit on sentence length has been exceeded.  (The case of production of language in a discourse context will be discussed separately.)

In this case, we focus on the process of producing a number-inflected verb after already having produced a subject NP of the form *The N1 Prep the N2*. This models the classic sentence completion paradigm of Bock & Miller (1991) and many subsequent experiments. The general finding is that people produce a more incorrect plural verbs following *the N1[sg] Prep the N2[pl]* than *the N1[sg] Prep the N2[sg]*.

## Grammar, representation, & dynamics
In a fuller model, the grammar is a collection of lexically anchored treelets. **The relation is not one of "less full" vs. "more full": rather, the treelet view is a different interpretation of the same model** ** *I don't fully understand this comment. What we're basically simulating here is a subset of the dimensions in the full treelet model, specifically just the number features on each treelet with some assumptions about which parse is forming in each case.* ** Each treelet consists of a mother and a finite number of daughters, some of which may be marked "optional". The mother and daughters are vectors of features, all of the same dimension, with the same interpretation assigned to each dimension. The harmony of a parse (which corresponds to an attractor at a harmony peak) is determined by the feature match between the attachment sites on linked treelets. Perfect feature match results in a harmony value of 1.0, with lower harmony values for feature mismatches and failures to attach required dependents.

To illustrate the dynamics of choosing a verb after having processed the subject NP, simplify the state space down to just three dimenions. The three dimensions represent the number markings on N1, N2, and the verb, with 0 coding singular (S) and 1 coding plural (P). Thus, each corner of the unit hypercube is a simple representation of a possible parse (including both grammatical and ungrammatical sequences of words). We define these points (which will be fixed points of the processing dynamics below). The following figure illustrates the state space: **I've put the figure in the figure directory but this command doesn't seem to be suitable for showing it...** ** *It seems to be working for me. Is there any error message for you, or just a blank space?* **

![title](Figures/AgrAttrStateSpace.pdf)

The size of the dot at a vertex represents the relative magnitude of its harmony value, as discussed next. The X marks the position of the initial state of the system for the simulations below, $\mathbf{s}_0$. The different conditions are modeled by choosing different initial conditions $\mathbf{s}_0$. $\mathbf{s}_0$ encodes what linguistic information has come before. $[0, 1, 0.5]$, e.g., is the starting point for choosing a verb after producing a singular N1 and a plural N2, and $[1, 0, 0.5]$ is the starting point for N1[pl]-N2[sg]. By starting the verb's number feature halfway between singular and plural, we model verb number selection by allowing the system to settle to one attractor or another given the previous input, the harmony landscape, and noise.

In [1]:
# Setting up an array of fixed points (centers of the summed Gaussians discussed below)
import numpy as np

centers = np.array([[0, 0, 0], # SSS
                   [0, 0, 1],  # SSP
                   [0, 1, 0],  # SPS
                   [0, 1, 1],  # SPP
                   [1, 0, 0],  # PSS
                   [1, 0, 1],  # PSP
                   [1, 1, 0],  # PPS
                   [1, 1, 1]])  # PPP
center_labels = ['SSS', 'SSP', 'SPS', 'SPP', 'PSS', 'PSP', 'PPS', 'PPP', 'other']

The harmony values for each of these vertices was determined in the following way. We assume that correct parses, i.e., treelets attaching in ways best licensed by the grammar with good feature matches, have the maximal harmony value $h_i$ of 1.0. For less-than-correct parses, we assume that partial parses (with otherwise good feature matches) are have much lower $h_i$ than correct parses, and that parses with a feature clash have the lowest $h_i$, i.e., $h_{i, correct} \gg h_{i, partial} \gg h_{i, clash}$. This amounts to saying that if the parser can't build a perfect parse, it strongly prefers a partial parse (a treelet *not* attaching anywhere) to a parse in which there is a feature clash. We chose to penalize partial parses by setting the relevant $h_i$s to 0.2 and feature clashes to 0.01, but the system seems to behave reasonably as long as the ordering is preserved.

We assume for the case at hand that the harmony values associated with each parse are the highest ones possible for that configuration:
- Correct parses (SSS, SPS, PSP, and PPP) have a harmony value of 1.0.
- For SPP and PSS, the parser prefers to *not* attach N1 (instead attaching a number-matching N2), resulting in an $h_i$ of 0.2.
- Finally, for SSP and PPS, the best option is to attach the N1 as subject despite its feature clash with the verb, making $h_i = 0.1$.

There are other ways of building each parse; e.g., PPS could result from N1 failing to attach, but that parse would be penalized twice, once for failure to attach and once for the feature clash between N2 (attached as the subject) and the verb. This would result in $h_i = 0.2 * 0.01 = 0.02$ (we assume the penalties for multiple violations are multiplied). Since there is a higher-harmony parse available (attaching N1 with feature clash), we assume that the parser chooses that higher-harmony parse instead.

In [14]:
# The order of the harmony values corresponds to the order of the centers in the last code block.
# ** To make this a document to share, just report one choice or characterize a range of behaviors.  
# ** Reporting precisely these two cases seems arbitrary

# produced LARGE amounts of attraction in the PS and PP conditions:
#harmony_values = np.array([1, 0.01, 1, 0.2, 0.2, 2, 0.01, 2])
# produced reasonable rates of attraction, but no sg./pl. asymmetry
harmony_values = np.array([1, 0.01, 1, 0.2, 0.2, 1, 0.01, 1])

The next step is to build the global harmony function, HF, out of the local harmony values. The global harmony function, $HF(\mathbf{s})$, is constructed by placing a Gaussian RBF, $\phi_i(\mathbf{s})$ at each local harmony locus $i$, scaled by the local harmony of that  $h_i$ (Muezzinoglu & Zurada, 2006):

$$HF(\mathbf{s}) = \sum_{i \in Centers} h_i\ \phi_i(\mathbf{s})$$

where 

$$\phi_i(\mathbf{s}) = \exp{\left(-\frac{\Vert\mathbf{s} - \mathbf{c}_i\Vert^2_2}{\gamma}\right)}$$

Here, $\mathbf{s}$ is the system state, i.e., the vector of activations of all the links and features. Each of these values ranges, in principle, over $(-\infty, \infty)$, but because of the design of the harmony function, the values will stay close the interval $[0, 1]$. $\mathbf{c}_i$ is the center of the $i$th Gaussian, each of which defines an attractor of the system dynamics below. $\gamma$ is a free parameter which specifies the widths of the RBFs, and thus determines how strongly the system prefers to achieve optimal parses (greater $\gamma$ means stronger preference for maximal harmony). $\Vert\cdot\Vert^2_2$ is the squared Euclidean distance (or the squared L2-norm).

The dynamics of the system are given by the gradient of the harmony + noise:

$$\dot{\mathbf{s}} = \nabla HF + \alpha\cdot\eta = -\frac{2}{\gamma}\sum_{i} h_i\ (\mathbf{s} - \mathbf{c}_i) \cdot \phi_i(\mathbf{s})\ + \ \alpha\cdot\eta$$

where $\dot{\mathbf{s}}$ denotes the change in the state variable with respect to time and $\eta$ is normally distributed noise. The magnitude, $\alpha$, of the noise is a free parameter.

By making the the system dynamics equal to the gradient of the harmony function, the system takes the steepest path possible to increase harmony (modulo the effect of the noise).

In the simplified system we are currently considering, $\mathbf{s}$ is a vector in $\mathbb{R}^3$, there are eight centers (given in `centers` above), and the harmony values of each center are given in `harmony_values`.

## Running the agreement attraction simulation
We can now run the simulation to test whether the simplified system. Here, the different conditions are modeled by setting the system at different initial states, as discussed above. We run the system until $\mathbf{s}$ is sufficiently close to an attractor or a maximum amount of time has passed. (Note: it takes about 10 minutes for the following code to run.)

In [15]:
# Parameters
tau = 0.1  # step size for the discretized (Euler forward) dynamics; this seems large...
maxsteps = 1000  # maximum number of timesteps before terminating integration
nruns = 500  # number of runs to do per condition
tol = 0.1  # how close the system has to be to an attractor to stop processing
alpha = 0.1  # noise magnitude
gamma = 0.1  # width of RBFs
ndim = centers.shape[1]  # number of dimensions in the state space

# Conditions, specified as s0 coordinates
conditions = np.array([[0, 1, 0.5],  # SP
                      [0, 0, 0.5],  # SS
                      [1, 0, 0.5],  # PS
                      [1, 1, 0.5]])  # PP
condition_labels = ['N1sg N2pl', 'N1sg N2sg', 'N1pl N2sg', 'N1pl N2pl']
ncond = len(condition_labels)

# Defining phi function
def phi(x, center, gamma):
    l2norm = np.linalg.norm(x - center)
    phi = np.exp(-l2norm**2 / gamma)
    return phi


# A function for updating the state of the system according to the negative
# gradient of the harmony function
def step_dyn(x, centers, harmonies, gamma):
    dx = np.zeros(x.shape)
    for c in range(centers.shape[0]):
        dx += harmonies[c] * (x - centers[c,]) * phi(x, centers[c,], gamma)
    return (-2./gamma)*dx


# Proximity to a fixed point
def not_close(x, centers, tol):
    for c in range(centers.shape[0]):
        l2norm = np.linalg.norm(x - centers[c,])
        if l2norm < tol:
            return False
        else:
            return True


# Singular or plural verb?
def sg_pl(x):
    x = np.round(x)
    if x[-1] == 0:
        return 0
    elif x[-1] == 1:
        return 1
    else:
        return -1


# Find out which fp. the system reached; not currently used!
def which_attr(x):
    x = np.round(x)
    for c in range(centers.shape[0]):
        if np.all(x == centers[c,]):
            return c
    return -1

# Doing the simulations
parses = np.zeros((nruns, ncond))
vnum = np.zeros((nruns, ncond))
for cond in range(ncond):
    print('\nCondition: {}'.format(condition_labels[cond]))
    print('Of {}: '.format(nruns), end='')
    for run in range(nruns):
        if (run+1) % 100 == 0:
            print('[{}] '.format(run+1), end='')
        x = np.zeros((maxsteps, ndim))
        x[0,] = conditions[cond,]
        noise = np.random.normal(0, 1, x.shape)

        t = 0
        while t < maxsteps-1:
            if not_close(x[t,], centers, tol = tol):
                x[t+1,] = x[t,] + tau*step_dyn(x[t,], centers, harmony_values, gamma) + alpha*noise[t,]
                t += 1
            else:
                break

        xtrunc = x[~np.all(x == 0, axis=1)]
        parses[run, cond] = which_attr(xtrunc[-1,])
        vnum[run, cond] = sg_pl(xtrunc[-1,])



Condition: N1sg N2pl
Of 500: [100] [200] [300] [400] [500] 
Condition: N1sg N2sg
Of 500: [100] [200] [300] [400] [500] 
Condition: N1pl N2sg
Of 500: [100] [200] [300] [400] [500] 
Condition: N1pl N2pl
Of 500: [100] [200] [300] [400] [500] 

In [16]:
# Now, just looking at the final distribution of parses:
parse_labels = ['Vsg', 'Vpl', 'other']
for cond in range(ncond):
    uniq1, cts1 = np.unique(parses[:,cond], return_counts=True)
    uniq2, cts2 = np.unique(vnum[:,cond], return_counts=True)
    print('\n{}:'.format(condition_labels[cond]))
    print('\tParse:\tProportion of runs:')
    print('\t{}'.format(''.join(['-']*27)))
    for u in range(len(uniq1)):
        if cts1[u] is not 0:
            print('\t{}:\t{}'.format(center_labels[int(uniq1[u])], cts1[u]/nruns))
    print('')
    for u in range(len(uniq2)):
        if cts2[u] is not 0:
            print('\t{}:\t{}'.format(parse_labels[int(uniq2[u])], cts2[u]/nruns))



N1sg N2pl:
	Parse:	Proportion of runs:
	---------------------------
	other:	0.008
	SSS:	0.054
	SPS:	0.88
	PSP:	0.01
	PPP:	0.048

	other:	0.006
	Vsg:	0.934
	Vpl:	0.06

N1sg N2sg:
	Parse:	Proportion of runs:
	---------------------------
	other:	0.006
	SSS:	0.968
	SPS:	0.004
	PSP:	0.016
	PPP:	0.006

	other:	0.006
	Vsg:	0.972
	Vpl:	0.022

N1pl N2sg:
	Parse:	Proportion of runs:
	---------------------------
	other:	0.026
	SSS:	0.042
	SPS:	0.022
	PSP:	0.878
	PPP:	0.032

	other:	0.016
	Vsg:	0.068
	Vpl:	0.916

N1pl N2pl:
	Parse:	Proportion of runs:
	---------------------------
	other:	0.022
	SSS:	0.018
	SPS:	0.024
	PSP:	0.026
	PPP:	0.91

	other:	0.02
	Vsg:	0.042
	Vpl:	0.938


The above tables show the proportion of runs that generated each type of parse in each condition. The first set of rows for each condition lists the actual parse formed, and the second set summarizes the results based on verb number only. There is a clear agreement attraction effect in the N1[sg] conditions, with N1[sg]-N2[pl] producing about 5-6% plural-verb parses and N1[sg]-N2[sg] producing about 2-3%, replicating the classic result.

However, a similar attraction effect is present in the N1[pl] conditions as well, with more singular agreement for N1[pl]-N2[sg] (~5-6%) than for N1[pl]-N2[pl] (~2-4%). Thus, this model fails to replicate the singular-plural asymmetry observed in English.

**Dependence on parameters and asymmetry:** I haven't explored this yet, other than the extra high harmony peaks for plural verb parses, which produced the sg./pl. asymmetry, but in the wrong direction. There was still agreement attraction in the N1[sg] cases, but for N1[pl], there were too many singular-verb parses for both N2[sg] (~26%) and N2[pl] (~13%). This is where I was unable to replicate the results of your Matlab script.